In [2]:
import os
from tqdm import tqdm
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from redisvl.extensions.llmcache import SemanticCache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Milvus


Create Embedding

In [4]:
embeddings = (
    OllamaEmbeddings(model="nomic-embed-text",base_url="http://localhost:11434")
)  

In [18]:
loader = PyPDFDirectoryLoader("./pdfs-master")
docs = loader.load()

In [19]:
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)   

print("Document splitting a in process")
docs = text_splitter.split_documents(docs)
print("Document splitted")
docs[1]

Document splitting a in process
Document splitted


Document(metadata={'source': 'pdfs-master\\NVIDIA - Accelerated Computing and the Democratization of Supercomputing Technical Overview (sc18-tesla-democratization-tech-overview-r4-web).pdf', 'page': 1}, page_content='ACCELERATED COMPUTING AND THE DEMOCRATIZATION OF SUPERCOMPUTING | TECHNICAL OVERVIEW | 2Table of Contents:\nThe Accelerated Data Center \nOptimizing Data Center Productivity\nSame Throughput with Fewer Server Nodes\nAren’t GPU-Accelerated Servers More Expensive? \nWhat About Operational Cost?\nWhat if the CPU is Given Away at No Cost?\nMaximizing Budget and Throughput\nLower Cost with  Acceleration\nDemocratizing the Supercomputer3\n6\n7\n8\n10\n11\n12\n13\n14')

In [ ]:
MILVUS_PORT="19530"
MILVUS_HOST="127.0.0.1"
 
print("start connecting to Milvus")
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"host": MILVUS_HOST ,"port": MILVUS_PORT,"db_name":"default"},
    collection_name="NvidiaPDF",
)

start connecting to Milvus


In [21]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

Connecting milvus for context retriver

In [5]:
MILVUS_PORT="19530"
MILVUS_HOST="127.0.0.1"
 
print("start connecting to Milvus")
vector_db = Milvus(
    embeddings,
    connection_args={"host": MILVUS_HOST ,"port": MILVUS_PORT,"db_name":"default"},
    collection_name="NvidiaPDF",
)

start connecting to Milvus


c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 0.3.0. An updated version of the class exists in the langchain-milvus package and should be used instead. To use it run `pip install -U langchain-milvus` and import as `from langchain_milvus import MilvusVectorStore`.
  warn_deprecated(


In [6]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [7]:
llmcache = SemanticCache(
    name="PDFs",
    prefix="PDFs",
    redis_url="redis://localhost:6379",
    distance_threshold=0.1
)
llmcache.set_threshold(0.1)

c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


22:15:14 redisvl.index.index INFO   Index already exists, not overwriting.


In [8]:
!rvl index listall --host localhost --port 6379

22:15:15 [RedisVL] INFO   Indices:
22:15:15 [RedisVL] INFO   1. PDFs


In [9]:
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler(
    public_key="pk-lf-bf97c47d-6b38-492a-a4e3-5faf52310023",
    secret_key="sk-lf-e0701d57-6d12-432b-926a-6b9c6bc4502a",
    host="http://localhost:3000",
    user_id="PDFs"
)

os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-e0701d57-6d12-432b-926a-6b9c6bc4502a"
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-bf97c47d-6b38-492a-a4e3-5faf52310023"
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

In [10]:
langfuse_handler = CallbackHandler()
langfuse = Langfuse()

langfuse_handler.auth_check()
langfuse.auth_check()

True

In [11]:
from semantic_router import Route

greeting_route = Route(
    name="greet_user",
    utterances=[
        "hello",
        "hi",
        "hey",
        "Whatsupppppp",
        "good morning",
        "good afternoon",
        "good evening",
    ],
)
#    time_route, 
routes = [
    greeting_route, 
]

In [12]:
from semantic_router import RouteLayer
from semantic_router.encoders import HuggingFaceEncoder
encoder = HuggingFaceEncoder()
name: str = "sentence-transformers/all-mpnet-base-v2"
type: str = "huggingface"
score_threshold: float = 0.3
encoder.score_threshold=0.3
encoder.name=name
print(f"Name: {encoder.name} ; Score: {encoder.score_threshold}")
rl = RouteLayer(encoder=encoder, routes=routes)

Name: sentence-transformers/all-mpnet-base-v2 ; Score: 0.3


In [13]:
print(f"Name: {encoder.name} ; Score: {encoder.score_threshold}")
def format_docs(docs):
    return "\n\n ------------".join(doc.page_content for doc in docs)

Name: sentence-transformers/all-mpnet-base-v2 ; Score: 0.3


In [14]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3",base_url="http://localhost:11434")


In [16]:
from langchain.chains import ConversationChain

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import  RunnableParallel, RunnablePassthrough
 
 
# Updated prompt template to include history and context
rag_template = """**Welcome to PDF Chat Assistant!**
 
I'm a Chatbot, your dedicated AI assistant for PDF summarization

 
**Please remember:**
 
* I value professionalism and confidentiality in our interactions. 
* If unsure, I will say I don't know and ask to give more information in the query. If the query is irrelevant, I will say it's beyond my area of expertise.

**Context:**
{context}
**Query:**
{question}
**Answer:**
"""
 
# Updating the prompt template
review_template = ChatPromptTemplate.from_template(rag_template)
 
# Creating the retriever with formatting
conversation_chain =(RunnableParallel(context=retriever | format_docs, question=RunnablePassthrough()) 
    | review_template
    | llm 
    | StrOutputParser())


In [22]:
from datetime import datetime
 
def get_time():
    now = datetime.now()
    return (
        f"The current time is {now.strftime('%H:%M')}"
    )

In [26]:
def semantic_layer(question: str):
    route = rl(question)
    if route.name == "get_time":
        question += f" (SYSTEM NOTE: {get_time()})"
    elif route.name == "greet_user":
        question += f" (SYSTEM NOTE: Say Hello I am PDF chatbot)"
    else:
        query1 = question + f" (SYSTEM NOTE: pdf query) "
        results = llmcache.check(prompt=question)
        if results:
            question = "###Cache hit: " + query1 + " " + results[0]["response"]
        else:
            response = conversation_chain.invoke(question, config={"callbacks": [langfuse_handler]})
            llmcache.store(prompt=question, response=response)
            question = "###Cache miss " + query1 + " " + response
    return question

In [27]:
question = """what is Nvidia DGX-1"""
 
sr_query = semantic_layer(question)
print(sr_query)

###Cache miss what is Nvidia DGX-1 (SYSTEM NOTE: pdf query)  The NVIDIA DGX-1 is a powerful system built with NVIDIA Pascal-powered Tesla P100 accelerators, featuring a massive increase in GPU memory capacity. It's designed for deep learning and AI-acceleration, with a complete software platform that includes major deep learning frameworks, GPU training systems, and GPU drivers.


In [30]:
docs = retriever.invoke(question)
context = "\n\n ------------".join(doc.page_content for doc in docs)
print(context)

For more information on NVIDIA DGX-1, visit www.nvidia.com/dgx1
© 2016 NVIDIA Corporation. All rights reserved. NVIDIA, the NVIDIA logo, and Iray are trademarks and/or registered trademarks of NVIDIA 
Corporation. All company and product names are trademarks or registered trademarks of the respective owners with which they are 
associated. Features, pricing, availability, and specifications are all subject to change without notice. DeC16computing for Infinite Opportunities 
The NVIDIA DGX-1 is the first system built with 
NVIDIA Pascal™-powered Tesla® P100 accelerators. 
The NVIDIA NVLink™ implementation delivers a 
massive increase in GPU memory capacity, giving you 
a system that can learn, see, and simulate our world.  
Analyze. Visualize. AI-Accelerate  
The NVIDIA DGX-1 software stack includes major 
deep learning frameworks, the NVIDIA DIGITS™ GPU 
training system, the NVIDIA Deep Learning SDK 
(e.g. CuDNN, NCCL), NVIDIA Docker, GPU drivers,

 ------------NVIDIA DGX-1 With Tesla 

In [29]:
import pandas as pd
from uptrain import EvalLLM, Evals, Settings
 
import time
 
def evaluate_uptrain(
    question, 
    context, 
    response, 
    langfuse_handler
):
    data = [{"question": question, "context": context, "response": response}]
    settings = Settings(
        model='ollama/llama3', 
    )
    eval_llm = EvalLLM(settings)
    res = eval_llm.evaluate(
        data=data,
        project_name="Pdf_chat",
        checks=[
            Evals.RESPONSE_COMPLETENESS, 
            Evals.RESPONSE_CONCISENESS, 
            Evals.RESPONSE_RELEVANCE,
            Evals.VALID_RESPONSE,
            Evals.RESPONSE_CONSISTENCY, 
            Evals.CONTEXT_RELEVANCE,
            Evals.RESPONSE_COMPLETENESS_WRT_CONTEXT,
            Evals.FACTUAL_ACCURACY,
            Evals.CODE_HALLUCINATION,
            Evals.PROMPT_INJECTION
        ]
    )
    trace = langfuse.trace(name="METRICS EVALUATION",user_id="PDFs")
    trace.span(name="retrieval", input={'question': question}, output={'context': context})
    trace.span(name="generation", input={'question': question, 'context': context}, output={'response': response})
    df = pd.DataFrame(res)
    for _, row in df.iterrows():
        for metric_name in [
            "response_completeness", "response_conciseness", "response_relevance", 
            "valid_response", "response_consistency", "response_completeness_wrt_context", 
            "factual_accuracy", "prompt_injection", "context_relevance", "factual_accuracy", 
            "code_hallucination"
        ]:
            trace.score(name=metric_name, value=res[0]["score_"+metric_name])
    return df

In [33]:
result_df = evaluate_uptrain(
    question=question, 
    context=context, 
    response=sr_query, 
    langfuse_handler=langfuse_handler
)

2024-07-13 23:15:49.372 | WARNING  | uptrain.operators.language.llm:fetch_responses:268 - Detected a running event loop, scheduling requests in a separate thread.
  0%|          | 0/1 [00:00<?, ?it/s]23:15:49 - LiteLLM:ERROR: ollama.py:499 - LiteLLM.ollama.py::ollama_acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
23:15:49 - LiteLLM:ERROR: main.py:411 - litellm.acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
Traceback (most recent call last):
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\main.py", line 390, in acompletion
    response = await init_response
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 505, in ollama_acompletion
    raise e
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 4


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



  0%|          | 0/1 [00:00<?, ?it/s]23:15:49 - LiteLLM:ERROR: ollama.py:499 - LiteLLM.ollama.py::ollama_acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
23:15:49 - LiteLLM:ERROR: main.py:411 - litellm.acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
Traceback (most recent call last):
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\main.py", line 390, in acompletion
    response = await init_response
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 505, in ollama_acompletion
    raise e
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 444, in ollama_acompletion
    raise OllamaError(status_code=resp.status, message=text)
litellm.llms.ollama.OllamaError: {"error":"model requires more system memory


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



  0%|          | 0/1 [00:00<?, ?it/s]23:15:49 - LiteLLM:ERROR: ollama.py:499 - LiteLLM.ollama.py::ollama_acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
23:15:49 - LiteLLM:ERROR: main.py:411 - litellm.acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
Traceback (most recent call last):
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\main.py", line 390, in acompletion
    response = await init_response
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 505, in ollama_acompletion
    raise e
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 444, in ollama_acompletion
    raise OllamaError(status_code=resp.status, message=text)
litellm.llms.ollama.OllamaError: {"error":"model requires more system memory

...

2024-07-13 23:15:51.333 | WARNING  | uptrain.operators.language.llm:fetch_responses:268 - Detected a running event loop, scheduling requests in a separate thread.
  0%|          | 0/1 [00:00<?, ?it/s]23:15:51 - LiteLLM:ERROR: ollama.py:499 - LiteLLM.ollama.py::ollama_acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
23:15:51 - LiteLLM:ERROR: main.py:411 - litellm.acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
Traceback (most recent call last):
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\main.py", line 390, in acompletion
    response = await init_response
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 505, in ollama_acompletion
    raise e
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 4


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



  0%|          | 0/1 [00:00<?, ?it/s]23:15:51 - LiteLLM:ERROR: ollama.py:499 - LiteLLM.ollama.py::ollama_acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
23:15:51 - LiteLLM:ERROR: main.py:411 - litellm.acompletion(): Exception occured - {"error":"model requires more system memory (5.9 GiB) than is available (5.6 GiB)"}
Traceback (most recent call last):
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\main.py", line 390, in acompletion
    response = await init_response
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 505, in ollama_acompletion
    raise e
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\litellm\llms\ollama.py", line 444, in ollama_acompletion
    raise OllamaError(status_code=resp.status, message=text)
litellm.llms.ollama.OllamaError: {"error":"model requires more system memory


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



1 validation error for ScoreBody
value
  none is not an allowed value (type=type_error.none.not_allowed)
Traceback (most recent call last):
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\langfuse\client.py", line 2226, in score
    request = ScoreBody(**new_dict)
              ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\pydantic\v1\main.py", line 341, in __init__
    raise validation_error
pydantic.v1.error_wrappers.ValidationError: 1 validation error for ScoreBody
value
  none is not an allowed value (type=type_error.none.not_allowed)
1 validation error for ScoreBody
value
  none is not an allowed value (type=type_error.none.not_allowed)
Traceback (most recent call last):
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\langfuse\client.py", line 2226, in score
    request = ScoreBody(**new_dict)
              ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\TUF GAMING\miniconda3\envs\rag\Lib\site-packages\pydantic\v